# Get File Names to be processed using glob

In [1]:
import glob
help(glob)

Help on module glob:

NAME
    glob - Filename globbing utility.

MODULE REFERENCE
    https://docs.python.org/3.9/library/glob
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

FUNCTIONS
    escape(pathname)
        Escape all special characters.
    
    glob(pathname, *, recursive=False)
        Return a list of paths matching a pathname pattern.
        
        The pattern may contain simple shell-style wildcards a la
        fnmatch. However, unlike fnmatch, filenames starting with a
        dot are special cases that are not matched by '*' and '?'
        patterns.
        
        If recursive is true, the pattern '**' will match any files and
        zero or more directories and subdirectories.
    
    ig

In [2]:
glob.glob("data/retail_db/**", recursive=True)

['data/retail_db\\',
 'data/retail_db\\categories',
 'data/retail_db\\categories\\part-00000',
 'data/retail_db\\create_db.sql',
 'data/retail_db\\create_db_tables_pg.sql',
 'data/retail_db\\customers',
 'data/retail_db\\customers\\part-00000',
 'data/retail_db\\departments',
 'data/retail_db\\departments\\part-00000',
 'data/retail_db\\load_db_tables_pg.sql',
 'data/retail_db\\orders',
 'data/retail_db\\orders\\part-00000',
 'data/retail_db\\order_items',
 'data/retail_db\\order_items\\part-00000',
 'data/retail_db\\products',
 'data/retail_db\\products\\part-00000',
 'data/retail_db\\README.md',
 'data/retail_db\\schemas.json']

In [3]:
glob.glob("data/retail_db/*/*")

['data/retail_db\\categories\\part-00000',
 'data/retail_db\\customers\\part-00000',
 'data/retail_db\\departments\\part-00000',
 'data/retail_db\\orders\\part-00000',
 'data/retail_db\\order_items\\part-00000',
 'data/retail_db\\products\\part-00000']

In [4]:
src_file_names = glob.glob("data/retail_db/*/part-*")
src_file_names

['data/retail_db\\categories\\part-00000',
 'data/retail_db\\customers\\part-00000',
 'data/retail_db\\departments\\part-00000',
 'data/retail_db\\orders\\part-00000',
 'data/retail_db\\order_items\\part-00000',
 'data/retail_db\\products\\part-00000']

In [5]:
import pandas as pd

In [6]:
for file_name in src_file_names:
    df = pd.read_csv(file_name, header = None)
    print(f'Shape of {file_name} is {df.shape}')

Shape of data/retail_db\categories\part-00000 is (58, 3)
Shape of data/retail_db\customers\part-00000 is (12435, 9)
Shape of data/retail_db\departments\part-00000 is (6, 2)
Shape of data/retail_db\orders\part-00000 is (68883, 4)
Shape of data/retail_db\order_items\part-00000 is (172198, 6)
Shape of data/retail_db\products\part-00000 is (1345, 6)


# Get Column Names using Schemas File

In [7]:
import json
import pandas as pd

In [8]:
def get_column_names(schemas, ds_name, sorting_key = "column_position"):
    column_details = schemas[ds_name]
    columns = sorted(column_details, key=lambda col: col[sorting_key])
    return [col["column_name"] for col in columns]

In [9]:
schemas = json.load(open("data/retail_db/schemas.json"))

In [10]:
order_columns = get_column_names(schemas, "orders")
orders = pd.read_csv("data/retail_db/orders/part-00000", names = order_columns)
orders

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE
...,...,...,...,...
68878,68879,2014-07-09 00:00:00.0,778,COMPLETE
68879,68880,2014-07-13 00:00:00.0,1117,COMPLETE
68880,68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT
68881,68882,2014-07-22 00:00:00.0,10000,ON_HOLD


# Getting Data Set Names from File Names/Paths using regex

In [12]:
src_file_names

['data/retail_db\\categories\\part-00000',
 'data/retail_db\\customers\\part-00000',
 'data/retail_db\\departments\\part-00000',
 'data/retail_db\\orders\\part-00000',
 'data/retail_db\\order_items\\part-00000',
 'data/retail_db\\products\\part-00000']

In [13]:
import re

In [16]:
for file in src_file_names:
    print(re.split('[/\\\]', file))

['data', 'retail_db', 'categories', 'part-00000']
['data', 'retail_db', 'customers', 'part-00000']
['data', 'retail_db', 'departments', 'part-00000']
['data', 'retail_db', 'orders', 'part-00000']
['data', 'retail_db', 'order_items', 'part-00000']
['data', 'retail_db', 'products', 'part-00000']


In [18]:
for file in src_file_names:
    print(f"Processing {file}")
    file_path_list = re.split('[/\\\]', file)
    ds_name = file_path_list[-2]
    columns = get_column_names(schemas, ds_name)
    df = pd.read_csv(file, names=columns)
    print(f"Shape of {ds_name} is {df.shape}")

Processing data/retail_db\categories\part-00000
Shape of categories is (58, 3)
Processing data/retail_db\customers\part-00000
Shape of customers is (12435, 9)
Processing data/retail_db\departments\part-00000
Shape of departments is (6, 2)
Processing data/retail_db\orders\part-00000
Shape of orders is (68883, 4)
Processing data/retail_db\order_items\part-00000
Shape of order_items is (172198, 6)
Processing data/retail_db\products\part-00000
Shape of products is (1345, 6)


# Generate File Paths for Target JSON Files Dynamically

In [21]:
base_dir = "data/retail_db_json"
for file in src_file_names:
    file_path_list = re.split('[/\\\]', file)
    ds_name = file_path_list[-2]
    file_name = file_path_list[-1]
    json_file_path = f'{base_dir}/{ds_name}/{file_name}'
    print(json_file_path)

data/retail_db_json/categories/part-00000
data/retail_db_json/customers/part-00000
data/retail_db_json/departments/part-00000
data/retail_db_json/orders/part-00000
data/retail_db_json/order_items/part-00000
data/retail_db_json/products/part-00000


# Writing Pandas Dataframe to JSON Files

In [22]:
import os

In [23]:
base_dir = "data/retail_db_json"
for file in src_file_names:
    print(f"Processing {file}")
    file_path_list = re.split('[/\\\]', file)
    ds_name = file_path_list[-2]
    file_name = file_path_list[-1]
    json_file_path = f'{base_dir}/{ds_name}/{file_name}'
    columns = get_column_names(schemas, ds_name)
    df = pd.read_csv(file, names=columns)
    os.makedirs(f'{base_dir}\{ds_name}', exist_ok=True)
    df.to_json(json_file_path, orient="records", lines=True)

Processing data/retail_db\categories\part-00000
Processing data/retail_db\customers\part-00000
Processing data/retail_db\departments\part-00000
Processing data/retail_db\orders\part-00000
Processing data/retail_db\order_items\part-00000
Processing data/retail_db\products\part-00000


# Modularize File Format Converter for Dataset

In [24]:
import glob
import json
import re
import pandas as pd

In [25]:
def get_column_names(schemas, ds_name, sorting_key = "column_position"):
    column_details = schemas[ds_name]
    columns = sorted(column_details, key=lambda col: col[sorting_key])
    return [col["column_name"] for col in columns]

In [26]:
def read_csv(file, schemas):
    file_path_list = re.split('[/\\\]', file)
    ds_name = file_path_list[-2]
    file_name = file_path_list[-1]
    columns = get_column_names(schemas, ds_name)
    df = pd.read_csv(file, names=columns)
    return df

In [27]:
def to_json(df, base_dir, ds_name, file_name):
    json_file_path = f'{base_dir}/{ds_name}/{file_name}'
    os.makedirs(f'{base_dir}\{ds_name}', exist_ok=True)
    df.to_json(json_file_path, orient="records", lines=True)

In [28]:
def file_converter(ds_name):
    src_base_dir = "data/retail_db"
    tgt_base_dir = "data/retail_db_json"

    schemas = json.load(open(f'{src_base_dir}/schemas.json'))
    files = glob.glob(f'{src_base_dir}/{ds_name}/part-*')

    for file in files:
        df = read_csv(file, schemas)
        file_name = re.split('[/\\\]', file)[-1]
        to_json(df, tgt_base_dir, ds_name, file_name)

In [29]:
ds_name = "orders"
file_converter(ds_name)

# Wrapper to Process all Data Sets

In [32]:
def file_converter(src_base_dir, tgt_base_dir, ds_name):
    schemas = json.load(open(f'{src_base_dir}/schemas.json'))
    files = glob.glob(f'{src_base_dir}/{ds_name}/part-*')

    for file in files:
        df = read_csv(file, schemas)
        file_name = re.split('[/\\\]', file)[-1]
        to_json(df, tgt_base_dir, ds_name, file_name)

In [33]:
def process_files(ds_names = None):
    src_base_dir = "data/retail_db"
    tgt_base_dir = "data/retail_db_json"
    schemas = json.load(open(f'{src_base_dir}/schemas.json'))
    if not ds_names:
        ds_names = schemas.keys()
    for ds_name in ds_names:
        print(f'Processing {ds_name}')
        file_converter(src_base_dir, tgt_base_dir, ds_name)

In [34]:
process_files(["orders", "order_items"])

Processing orders
Processing order_items


In [35]:
process_files()

Processing departments
Processing categories
Processing orders
Processing products
Processing customers
Processing order_items
